In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Intro to Google Agentspace

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/search/agentspace/intro_agentspace.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fsearch%2Fagentspace%2Fintro_agentspace.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/search/agentspace/intro_agentspace.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/search/agentspace/intro_agentspace.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/53/X_logo_2023_original.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/notebook_template.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| | |
|-|-|
| Author(s) | [Dave Wang](https://github.com/wadave/) |

# Overview
[Google Agentspace](https://cloud.google.com/products/agentspace?e=48754805) is a powerful AI-powered search and productivity tool designed for businesses. It acts as a central hub for finding information and getting things done within an organization. It unlocks enterprise expertise for employees with agents that bring together Gemini’s advanced reasoning, Google-quality search, and enterprise data, regardless of where it’s hosted.

Agentspace is currently GA with allowlist. You can sign up for [early access](https://cloud.google.com/resources/google-agentspace).

Acceptance is on a rolling basis and you will receive an email for acceptance when we assign a tech onboarding lead to your account. 

This notebook demonstrates how to use the Agentspace Client libraries to access the search engine.

There are several ways to interact with the Agentspace engine:

- 1. Search Method

- 2. Answer Method


**Reference:**

- https://cloud.google.com/agentspace/agentspace-enterprise/docs/libraries#client-libraries-usage-python
- https://cloud.google.com/agentspace/agentspace-enterprise/docs/reference/rest

## Get Started

Install the following packages required to execute this notebook.


In [ ]:
%pip install --upgrade --quiet google-cloud-discoveryengine

### Restart runtime

To use the newly installed packages in this Jupyter runtime, you must restart the runtime. You can do this by running the cell below, which restarts the current kernel.

The restart might take a minute or longer. After it's restarted, continue to the next step.

In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>

# Prerequisites
- You need to follow [this link](https://cloud.google.com/agentspace/agentspace-enterprise/docs/quickstart-agentspace) to set up datastore and engine first.
- Make a note of your Agentspace app engine id (aka, app id)

### Authenticate your notebook environment (Colab only)

If you're running this notebook on Google Colab, run the cell below to authenticate your environment.

In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run gcloud config list.
* Run gcloud projects list.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [5]:
# Use the environment variable if the user doesn't provide Project ID.
import os
LOCATION = "global"

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))


## Import libraries

In [4]:
import pprint

from google.api_core.client_options import ClientOptions
from google.cloud import discoveryengine_v1 as discoveryengine


# 1. Search method

In [5]:
project_id = PROJECT_ID
location = LOCATION  
engine_id = "[your-Agentspace-app-id]" 

In [10]:
def get_search_sdk_response(
    project_id: str,
    location: str,
    engine_id: str,
    search_query: str,
    prompt_preamble: str,
) -> list[discoveryengine.SearchResponse]:
    """Retrieves search results from the Discovery Engine using the Python SDK.

    This function queries the specified Discovery Engine with the provided search
    query and configuration options.  It supports specifying a location-specific
    endpoint for the API.

    Args:
        project_id: The Google Cloud project ID.
        location: The location of the Discovery Engine (e.g., "global", "us-central1").
        engine_id: The ID of the Discovery Engine.
        search_query: The search query string.
        prompt_preamble: The preamble to use for search summaries.

    Returns:
        A list of `discoveryengine.SearchResponse` objects containing the search results.
        This list may be empty if no results are found.

    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store

    """

    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    # Create a client
    client = discoveryengine.SearchServiceClient(client_options=client_options)

    # The full resource name of the search app serving config
    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_config"

    # Optional - only supported for unstructured data: Configuration options for search.
    # Refer to the `ContentSearchSpec` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest.ContentSearchSpec
    content_search_spec = discoveryengine.SearchRequest.ContentSearchSpec(
        # For information about snippets, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/snippets
        snippet_spec=discoveryengine.SearchRequest.ContentSearchSpec.SnippetSpec(
            return_snippet=True
        ),
        # For information about search summaries, refer to:
        # https://cloud.google.com/generative-ai-app-builder/docs/get-search-summaries
        summary_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec(
            summary_result_count=5,
            include_citations=True,
            ignore_adversarial_query=True,
            ignore_non_summary_seeking_query=True,
            model_prompt_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelPromptSpec(
                preamble=prompt_preamble
            ),
            model_spec=discoveryengine.SearchRequest.ContentSearchSpec.SummarySpec.ModelSpec(
                version="preview",
            ),
        ),
        # -------extractive content-----
        extractive_content_spec=discoveryengine.SearchRequest.ContentSearchSpec.ExtractiveContentSpec(
            max_extractive_segment_count=2,
            max_extractive_answer_count=2,
            return_extractive_segment_score=True,
        ),
    )

    # Refer to the `SearchRequest` reference for all supported fields:
    # https://cloud.google.com/python/docs/reference/discoveryengine/latest/google.cloud.discoveryengine_v1.types.SearchRequest
    request = discoveryengine.SearchRequest(
        serving_config=serving_config,
        query=search_query,
        page_size=10,
        content_search_spec=content_search_spec,
        query_expansion_spec=discoveryengine.SearchRequest.QueryExpansionSpec(
            condition=discoveryengine.SearchRequest.QueryExpansionSpec.Condition.AUTO,
        ),
        spell_correction_spec=discoveryengine.SearchRequest.SpellCorrectionSpec(
            mode=discoveryengine.SearchRequest.SpellCorrectionSpec.Mode.AUTO
        ),
    )

    response = client.search(request)
    # print(response)

    return response

In [11]:
search_query = "who founded cymbal bank?"
prompt_preamble = " "

In [12]:
response = get_search_sdk_response(
    project_id,
    location,
    engine_id,
    search_query,
    prompt_preamble,
)

In [13]:
response.summary

In [14]:
from google.protobuf.json_format import MessageToDict

document_dict = MessageToDict(
    response.results[0].document._pb, preserving_proto_field_name=True
)

In [15]:
document_dict["derived_struct_data"]["extractive_answers"][0]["content"]

# 2. Answer method

In [21]:
def get_answer_sdk_response(
    project_id: str,
    location: str,
    engine_id: str,
    query: str,
) -> discoveryengine.AnswerQueryResponse:
    """Retrieves an answer from the Discovery Engine using the AnswerQuery API.

    This function queries the specified Discovery Engine with the provided query
    and configuration options using the `AnswerQuery` API. It supports
    specifying a location-specific endpoint for the API.

    Args:
        project_id: The Google Cloud project ID.
        location: The location of the Discovery Engine (e.g., "global", "us-central1").
        engine_id: The ID of the Discovery Engine.
        query: The query string.

    Returns:
        A `discoveryengine.AnswerQueryResponse` object containing the answer,
        citations, and other related information.

    #  For more information, refer to:
    # https://cloud.google.com/generative-ai-app-builder/docs/locations#specify_a_multi-region_for_your_data_store
    """
    client_options = (
        ClientOptions(api_endpoint=f"{location}-discoveryengine.googleapis.com")
        if location != "global"
        else None
    )

    # Create a client
    client = discoveryengine.ConversationalSearchServiceClient(
        client_options=client_options
    )

    # The full resource name of the Search serving config
    serving_config = f"projects/{project_id}/locations/{location}/collections/default_collection/engines/{engine_id}/servingConfigs/default_serving_config"

    # Optional: Options for query phase
    # The `query_understanding_spec` below includes all available query phase options.
    # For more details, refer to https://cloud.google.com/generative-ai-app-builder/docs/reference/rest/v1/QueryUnderstandingSpec
    query_understanding_spec = discoveryengine.AnswerQueryRequest.QueryUnderstandingSpec(
        query_rephraser_spec=discoveryengine.AnswerQueryRequest.QueryUnderstandingSpec.QueryRephraserSpec(
            disable=False,  # Optional: Disable query rephraser
            max_rephrase_steps=1,  # Optional: Number of rephrase steps
        ),
        # Optional: Classify query types
        query_classification_spec=discoveryengine.AnswerQueryRequest.QueryUnderstandingSpec.QueryClassificationSpec(
            types=[
                discoveryengine.AnswerQueryRequest.QueryUnderstandingSpec.QueryClassificationSpec.Type.ADVERSARIAL_QUERY,
                discoveryengine.AnswerQueryRequest.QueryUnderstandingSpec.QueryClassificationSpec.Type.NON_ANSWER_SEEKING_QUERY,
            ]  # Options: ADVERSARIAL_QUERY, NON_ANSWER_SEEKING_QUERY or both
        ),
    )

    # Optional: Options for answer phase
    # The `answer_generation_spec` below includes all available query phase options.
    # For more details, refer to https://cloud.google.com/generative-ai-app-builder/docs/reference/rest/v1/AnswerGenerationSpec
    answer_generation_spec = discoveryengine.AnswerQueryRequest.AnswerGenerationSpec(
        ignore_adversarial_query=False,  # Optional: Ignore adversarial query
        ignore_non_answer_seeking_query=False,  # Optional: Ignore non-answer seeking query
        ignore_low_relevant_content=False,  # Optional: Return fallback answer when content is not relevant
        model_spec=discoveryengine.AnswerQueryRequest.AnswerGenerationSpec.ModelSpec(
            model_version="gemini-1.5-flash-001/answer_gen/v2",  # Optional: Model to use for answer generation
        ),
        prompt_spec=discoveryengine.AnswerQueryRequest.AnswerGenerationSpec.PromptSpec(
            preamble="Give a detailed answer.",  # Optional: Natural language instructions for customizing the answer.
        ),
        include_citations=True,  # Optional: Include citations in the response
        answer_language_code="en",  # Optional: Language code of the answer
    )

    # Initialize request argument(s)
    request = discoveryengine.AnswerQueryRequest(
        serving_config=serving_config,
        query=discoveryengine.Query(text=query),
        session=None,  # Optional: include previous session ID to continue a conversation
        query_understanding_spec=query_understanding_spec,
        answer_generation_spec=answer_generation_spec,
    )

    # Make the request
    response = client.answer_query(request)

    # Handle the response
    print(response)

    return response

In [22]:
query = "who founded cymbal bank?"
resp = get_answer_sdk_response(
    project_id,
    location,
    engine_id,
    query,
)

In [23]:
resp.answer.answer_text

## Cleaning up



In [2]:
# Delete search engine and datastore
def delete_search_engine(project_id: str, location: str, data_store_id: str, search_engine_id: str):
    """Deletes a Discovery Engine Search Engine.

    Args:
        project_id: The Google Cloud project ID.
        location: The location of the Data Store (e.g., "global").
        data_store_id: The ID of the Data Store containing the Search Engine.
        search_engine_id: The ID of the Search Engine to delete.
    """

    client = discoveryengine.SearchEngineServiceClient()

    # Construct the Search Engine name.
    search_engine_name = client.search_engine_path(
        project=project_id,
        location=location,
        data_store=data_store_id,
        search_engine=search_engine_id,
    )

    try:
        # Delete the Search Engine.
        client.delete_search_engine(name=search_engine_name)
        print(f"Search Engine '{search_engine_id}' deleted successfully.")

    except Exception as e:
        print(f"Error deleting Search Engine: {e}")


data_store_id = "[your-data-store-id]"  # Replace with your Data Store ID
delete_search_engine(project_id, location, data_store_id, engine_id)